In [ ]:
import pandas as pd
import altair as alt

In [ ]:
pip install xlrd xlwt openpyxl

In [ ]:
df.columns

In [ ]:
df = pd.read_excel('2014-2020.xlsx', usecols=range(0,12) )
df

In [ ]:
df[['Time','Date']] = df['Час/Дата'].str.split("-",expand=True)

In [ ]:
df[['Day']] = df['Date'].str[:5]
df[['Year']] = df['Date'].str[6:]
df[['Month']] = df['Date'].str[3:5]
df

In [ ]:
df[['Season']] = 0
df.loc[df['Month'].isin(['12', '01','02']), 'Season'] = 'winter'
df.loc[df['Month'].isin(['03', '04','05']), 'Season'] = 'spring'
df.loc[df['Month'].isin(['06', '07','08']), 'Season'] = 'summer'
df.loc[df['Month'].isin(['09', '10','11']), 'Season'] = 'fall'
df

In [ ]:
aes = df[['AES','Time','Date', 'Day', 'Year']]
aes.loc[:, 'Name'] = "AES"
aes.rename({'AES': 'All'}, axis=1,inplace=True)

tec = df[['TEC','Time','Date', 'Day', 'Year']]
tec.loc[:, 'Name'] = "TEC"
tec.rename({'TEC': 'All'}, axis=1,inplace=True)           

vde = df[['VDE','Time','Date', 'Day', 'Year']]
vde.loc[:, 'Name'] = "VDE"
vde.rename({'VDE': 'All'}, axis=1,inplace=True)
           
tes = df[['TES','Time','Date', 'Day', 'Year']]
tes.loc[:, 'Name'] = "TES"
tes.rename({'TES': 'All'}, axis=1,inplace=True)

ges = df[['GES','Time','Date', 'Day', 'Year']]
ges.loc[:, 'Name'] = "GES" 
ges.rename({'GES': 'All'}, axis=1,inplace=True)
           
gaes = df[['GAES_GEN','Time','Date', 'Day', 'Year']]
gaes.loc[:, 'Name'] = "GAES"
gaes.rename({'GAES_GEN': 'All'}, axis=1,inplace=True)

electricity_df = pd.concat([aes, tec, vde, tes, ges, gaes])

In [ ]:
alt.data_transformers.disable_max_rows()

1) Як змінювалась структура генерації електроенергії за роками?

Я обрала Area Graph для того, щоб показати зміну структури генерації електроенергії за роками. Такий підхід дозволяє зручно візуалізувати зміну в часі та частку в цілому. Альтернативний спосіб - використати Line chart, проте таким чином, ми не зможемо проаналізувати зміни генерації електроенергії загалом.  

In [ ]:
electricity_df

In [ ]:
alt.Chart(electricity_df).transform_filter(alt.FieldRangePredicate(field='Year', range=[2014, 2020])).mark_area().encode(
    x = alt.X('Year:N'),
    y = alt.Y('All:Q', aggregate = 'sum'),
    color = alt.Color('Name:N'),
    order = alt.Order('All:Q', aggregate='sum', sort = 'ascending')
).properties(width = 950, height = 500, background = '#F9F9F9', padding = 25)

2) Як залежить споживання електроенергії від дня року та години доби?

Для цієї задачі, я обрала Heatmap, оскільки це один з найзручніших способів показати взаємозв'язок та кореляцію між змінними. Також, це було зручним рішенням, оскільки потрібно було показати залежність і від дня року, і від години. Таким чином, колір прямокутника демонструє величину споживання, вісь x - день року, а вісь y - годину доби. Єдиним недоліком такої візуалізації є її громіздкість.   

In [ ]:
alt.Chart(df).mark_rect().encode(
    x = alt.X('Day:N'),
    y = alt.Y('Time:N', sort =  alt.Sort(field = 'Time', op = 'count', order = 'ascending')),
    color = alt.Color('CONSUMPTION:Q')).properties( background = '#F9F9F9', padding = 25)

3) Як змінюється генерація електроенергії з різних джерел впродовж доби?

Я обрала bar chart, щоб дати відповідь на поставлене питання. Кожен стовпчик поділений на 6 частин, кожна з яких означає певне джерело електроенергії. Перевага такого підходу у зручності та інтуїтивності считування результатів. Альтернативний спосіб - візуалізація за допомогою heatmap.   

In [ ]:
alt.Chart(electricity_df).mark_bar().encode(
    x = alt.X('Time:N', sort = alt.Sort(field = 'Time', op = 'count', order = 'descending')),
    y = alt.Y('All:Q', aggregate = 'sum'),
    color = alt.Color('Name:N'),
    tooltip = alt.Tooltip('All:Q', aggregate = 'sum')
).properties(width = 950, height = 500)


In [ ]:
df["Time"]=pd.to_numeric(df["Time"])

4) Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

Щоб показати зміну у розрізі місяців та пір року, я обрала варіант візуалізації Heatmap, розділений на 4 частини (= 4 пори року). В кожній секції, що означає пору року, є прямокутники, колір яких означає розмір споживання електроенергії. Перевагою цього підходу є те, що легко зрозуміти залежність такої великої кількості змінних.   

In [ ]:
alt.Chart(df).mark_rect().encode(
    x = alt.X('Month:N'),
    y = alt.Y('Time:N',sort = alt.Sort(field = 'Time', op = 'count', order = 'ascending')),
    color = alt.Color('CONSUMPTION:Q')).properties(width = 750/2, height = 500/2).facet(facet = alt.Facet('Season:N'), columns = 2)

5) Як змінюється споживання електроенергії впродовж тижня?

Для відповіді на це питання, я обрала bar chart, кожен стовпець означає день тижня. Перевагою цього способу є зручність візуального сприйняття зміни в часті. Альтернативний варіант -  line chart або area graph, проте у цьому випадку нетак легко прослідковувалась зміна по днях. 

In [ ]:
weekday = pd.DatetimeIndex(list(df['Date']))
df[['DayName']] = weekday.day_name()
df

In [ ]:
alt.Chart(df).mark_bar().encode(
    x = alt.X('DayName:N', sort = alt.Sort(field = 'CONSUMPTION', op = 'mean', order = 'ascending')),
    y = alt.Y('CONSUMPTION:Q', aggregate = 'mean'),
    tooltip = alt.Tooltip('CONSUMPTION:Q',aggregate = 'mean')
).properties(width = 950, height = 500)